In [ ]:
HTML(read(open("style.css"), String))

# Engine Games

This notebook contains functions for 2 engines to play against each other. 

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("NBInclude")
using NBInclude

# Imports

In [ ]:
@nbinclude("AdvancedBoard.ipynb")

## Engines

In [ ]:
@nbinclude("RandomChess.ipynb")

In [ ]:
@nbinclude("Minimax.ipynb")

In [ ]:
@nbinclude("AlphaBetaPruning.ipynb")

In [ ]:
@nbinclude("IterativeDeepening.ipynb")

## Utility

In [ ]:
@nbinclude("Utility.ipynb")

In [ ]:
@nbinclude("PGN_Export.ipynb")

In [ ]:
@nbinclude("PGN_Import.ipynb")

## Players

The `playWhiteMove` function takes in 4 arguments.
1. `game` is the current state of type `Game`
1. `current_boardscore` is the static evaluation of the position from the perspective of the moving player
1. `current_hash` is the hash of the current position
1. `cache`  is a dictionary which stores the calculated values

The function makes a move for the white player and returns the new score and hash of the position.

In [ ]:
function playWhiteMove(game, aBoard::AdvBoard, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    starttime = time()
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(aBoard, currentcurrent_boardscorescore, 4, cache)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(aBoard, 4, cache)

    # ITERATIVE DEEPENING ENGINE
    eval, move = iterativeDeepening(aBoard, 5, cache)

    #----------------------------------------------------------------
    clearCache(board(game), move, cache)
    domoveAdv!(aBoard, move)
    println("Time Engine needs to be calculated: $(time() - starttime )")
    println("Evaluation of engine: $eval")
    println("Current board score: $(aBoard.score)")
    println("Current entries in Cache: $(length(cache))")
    display(board(game))
    return aBoard
end

The `playBlackMove` function takes in 4 arguments.
1. `game` is the current state of type `Game`
1. `current_boardscore` is the static evaluation of the position from the perspective of the moving player
1. `current_hash` is the hash of the current position
1. `cache`  is a dictionary which stores the calculated values

The function makes a move for the black player and returns the new score and hash of the position.

In [ ]:
function playBlackMove(game, aBoard::AdvBoard, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    starttime = time()
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(aBoard, currentcurrent_boardscorescore, 4, cache)
    
    # ALPHA BETA PRUNING ENGINE
    eval, move = alphaBetaPruning(aBoard, 4, cache)
    # ITERATIVE DEEPENING ENGINE
    # eval, move = iterativeDeepening(aBoard, 2, cache)
    
    #----------------------------------------------------------------
    
    clearCache(board(game), move, cache)
    domoveAdv!(aBoard, move)
    println("Time Engine needs to be calculated: $(time() - starttime )")
    println("Evaluation of engine: $eval")
    println("Current board score: $(aBoard.score)")
    println("Current entries in Cache: $(length(cache))")
    display(board(game))
    return aBoard
end

## Game

The `playGame` function takes in optional 1 arguments. 
1. `pgnFile` is optional and default is nothing. 'pgnFile' is the name of the pgn file to be opened. If nothing is passed, the function will generate a new game.

The function plays a game between the white and black engines and returns the final state of the game.

In [ ]:
function playGame(pgnFile=nothing)
    # Setup Board
    if pgnFile == nothing
        # new Game
        println("Initializing new Game...")
        game = Game()
        display(board(game))
        setGameHeaders(game)
    else
        # continue saved game
        println("Opening $(pgnFile) ...")
        game = openPGNtoGame(pgnFile)
        if game == nothing
            println("No such file found")
            return
        end
        toend!(game)
        display(game)
    end   
    cacheWhite = initCache()
    cacheBlack = initCache()
    moveCount = 1
    aBoard = AdvBoard(board(game))
    while true
        # White Move
        
        println("Move $(moveCount)")
        println("White")
        
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                aBoard = playWhiteMove(game,aBoard, cacheWhite)
            end
        else
            break
        end
        
        # Black Move
        println("Black")
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                aBoard = playBlackMove(game,aBoard, cacheBlack)
            end
        else
            break
        end
        moveCount += 1
    end
    setWin(game)
    saveGameToPGN(game)
    return game
end

In [ ]:
playGame()